To index data in Azure SQL db, for now we would need to use Azure AI search. 



#### First, let's populate the table with sample data:

1. Update your .env file with required environment variables for connecting to your Azure SQL database (assuming you already have created a Azure SQL resource and have created a database)
2. Ensure you are in the src folder and run: python init_sql_db.py
3. Above step will populate a table with sample products and their descriptions

#### Now, assuming you have created an Azure AI Search resource (free version is sufficient):
1. In you Azure AI search resource, go to 'Import and Vectorize Data -> choose/connect your Azure SQL Database under 
2. Populate required info to connect to your SQL database 
3. Click on 'Test connection' to validate
4. Now you can choose the product_catalogue table 
5. After connecting, you can now choose the column to vectorize, and the Azure Open AI service to use to generate embeddings.
6. After this is done, it will also create the index.




#### update .env values:
- Ensure to populate the AZURE_SEARCH_ENDPOINT AND AZURE_SEARCH_KEY variables.
    - endpoint is found  under url value
    - you can find the key value under settings -> key
- Also put update the index name.

#### Now we are ready to perform a simple test:

##### Connecting to AI search client:

In [17]:
import os
from dotenv import load_dotenv
load_dotenv()
from azure.search.documents import SearchClient  
from azure.core.credentials import AzureKeyCredential 

class AzureSearch():
    def __init__(self):
        self.aisearch_key = os.environ.get("AZURE_SEARCH_KEY")
        self.service_endpoint = os.environ.get("AZURE_SEARCH_ENDPOINT")
        self.index_name = os.environ.get("AZURE_SEARCH_INDEX")
        self.embedding_length = 1536
        self.search_client = SearchClient(self.service_endpoint, self.index_name, credential=AzureKeyCredential(self.aisearch_key))

    async def search_query(self, text_input):
        results = self.search_client.search(search_text=text_input,
                                                    select=[
                                            "chunk_id", "text_vector","name"],
                                         top=5)
        print("Results were retrieved successfully.")
        for r in results:
            print(f"Chunk ID: {r['chunk_id']},Name: {r['name']}, Text Vector: {r['text_vector']}")
        return results

In [18]:
s = AzureSearch()

In [19]:
question = "I want a smart watch that is advanced, can track my health, has a long battery life and is waterproof"

In [20]:
res = await s.search_query(question)

Results were retrieved successfully.
Chunk ID: 885d04b10227_743_pages_0,Name: Apple Watch Ultra, Text Vector: [-0.004714222, 0.0061267987, 0.00015006517, -0.017221272, 0.0021611075, 0.0074210977, -0.030008812, -0.00896885, 0.0034807518, -0.017653832, 0.013334996, 0.022749923, -0.007367028, -0.023195999, -0.027791943, -0.018181013, 0.021655006, 0.014355566, 0.011138406, -0.016518364, -0.019776076, 0.0024618714, 0.010394944, -0.0023452833, -0.015099028, -0.01530179, 0.008563325, -0.011956213, 0.029927706, -0.023182482, 0.0076914476, -0.0020681748, -0.031793118, -0.015761385, -0.0019025856, 0.023547454, -0.012287391, -0.012686157, 0.027981188, -0.01611284, 0.008793122, 0.009705552, -0.0016601158, 0.011462825, -0.025683217, 0.028413748, -0.010895091, -0.008252423, -0.0010873125, 0.0031208487, -0.011753451, 0.0018485157, -0.00065771, -0.0065728757, 0.0018941372, -0.006062591, -0.0044607692, 0.018235084, 0.019316481, 0.0031225383, 0.0017082718, 0.0017167203, -0.025710251, 0.031198349, -0.005